In [ ]:
import pandas as pd
import pylab as plt
import lib_search_dispersion, lib_util
from matplotlib.colors import LogNorm
import numpy as np
from lib_util import find_moving_average
from lib_search_dispersion import clean_Eic
from scipy.signal import find_peaks, savgol_filter
%matplotlib inline

In [ ]:
df_rev = pd.read_csv('data/reverse_search_F16.csv', parse_dates=['start_time', 'end_time'])
df_rev[(df_rev.start_time.dt.month == 8)&(df_rev.start_time.dt.day==29)]

In [ ]:
df_5y = pd.read_csv('data/training_data_5y_F16.csv',  parse_dates=['start_time', 'end_time'])
df_5y[(df_5y.start_time.dt.month == 7)&(df_5y.start_time.dt.day==9)]

In [ ]:
df_dd = pd.concat([
    df_rev.iloc[[512]],
    df_5y.iloc[[6]],
    #df_5y.iloc[[8]],
    df_5y.iloc[[482]],
    df_5y.iloc[[542]],
    df_5y.iloc[[548]],

    #df_5y.iloc[[203]]  # triple dispersion
])
df_dd

In [ ]:
def plot_spec(fh, start_time, end_time, method='dual_peaks', axvline_idx=None):
    i = fh['t'].searchsorted(start_time)
    j = fh['t'].searchsorted(end_time)
        
    delta = j - i
    i -= int(delta * 0.5)
    j += int(delta * 0.5)
    
    fig, axes = plt.subplots(2, 1, figsize=(12, 4), dpi=500)
    
    # Ion Flux Spectrogra ------------------------
    im = axes[0].pcolor(
        fh['t'][i:j],
        np.log10(fh['ch_energy']),
        fh['ion_d_ener'][:, i:j], 
        norm=LogNorm(vmin=1e3, vmax=1e8), cmap='jet'
    )
    plt.colorbar(im, ax=axes[0]).set_label('Energy Flux')
    axes[0].set_ylabel('Ion Energy')
    
    if method == 'dual_peaks':
        t_p, lower_Ep, upper_Ep = calculate_dual_peaks(fh, i, j) 
        axes[0].plot(t_p, lower_Ep, 'b*-')
        axes[0].plot(t_p, upper_Ep, 'g*-')
    elif method == 'line_resets':
        t_lines, lines = calculate_line_resets(fh, i, j)
        for line_E in lines:
            axes[0].plot(t_lines, line_E, 'b*-')
    
    if axvline_idx:
        axes[0].axvline(t_p[axvline_idx])
    
    # Electron Flux Spectrogram ------------------------
    im = axes[1].pcolor(
        fh['t'][i:j],
        np.log10(fh['ch_energy']),
        fh['el_d_ener'][:, i:j], 
        norm=LogNorm(vmin=1e5, vmax=1e10), cmap='jet'
    )
    plt.colorbar(im, ax=axes[1]).set_label('Energy Flux')
    axes[1].set_ylabel('Electron Energy')
    

In [ ]:
def calculate_dual_peaks(fh, start_i, stop_j):
    t = fh['t']
    E = fh['ch_energy']
    lower_Ep = np.zeros(stop_j - start_i, dtype=float)
    upper_Ep = np.zeros(stop_j - start_i, dtype=float)
    lower_Flux = np.zeros(stop_j - start_i, dtype=float)
    upper_Flux = np.zeros(stop_j - start_i, dtype=float)
    
    last_filled = 'lower'
    
    for ii, i in enumerate(range(start_i, stop_j)):
        data = fh['ion_d_ener'][:, i]
        smoothed = savgol_filter(data, 5, 2)
        peaks = find_peaks(smoothed)[0]

        peaks = np.array([peak for peak in peaks if data[peak] > 10**6.5 and np.log10(E[peak]) < 3.5])
        
        if len(peaks) == 0:
            lower_Ep[ii] = np.nan
            lower_Flux[ii] = np.nan
            upper_Ep[ii] = np.nan
            upper_Flux[ii] = np.nan
        
        if len(peaks) == 1:
            Enext = E[peaks[0]]
            Fluxnext = data[peaks[0]]
                
            if ((ii == 0 or
                (np.isnan(lower_Ep[ii-1]) and np.isnan(upper_Ep[ii-1])) or   
                (np.isfinite(lower_Ep[ii-1]) and lower_Ep[ii-1] >= Enext) or
                np.isnan(upper_Ep[ii-1])) and last_filled in ('lower', 'both')):
                lower_Ep[ii] = Enext
                lower_Flux[ii] = Fluxnext
                upper_Ep[ii] = np.nan
                upper_Flux[ii] = np.nan
                last_filled = 'lower'                
            else:
                lower_Ep[ii] = np.nan
                lower_Flux[ii] = np.nan
                upper_Ep[ii] = Enext
                upper_Flux[ii] = Fluxnext
                last_filled = 'upper'
        elif len(peaks) >= 2:
            # Pick peaks with top-2 fluxes
            I = np.argsort(smoothed[peaks])[-2:]
            low_E, hi_E = sorted(E[peaks[I]])
            low_Flux, hi_Flux = data[peaks][np.argsort(E[peaks[I]])]
            
            # if high is closer to last lower line, then probably a false signal
            if np.abs(hi_E - lower_Ep[ii-1]) < np.abs(low_E - lower_Ep[ii-1]):
                lower_Ep[ii] = hi_E
                upper_Ep[ii] = np.nan
                last_filled = 'lower'
            else:
                lower_Ep[ii], upper_Ep[ii] = low_E, hi_E
                lower_Flux[ii], upper_Flux[ii] = low_Flux, hi_Flux
                last_filled = 'both'
            
    lower_Ep = np.log10(lower_Ep)
    upper_Ep = np.log10(upper_Ep)
    
    for i in range(1, lower_Ep.size - 1):
        isfin1 = np.isfinite(lower_Ep[i-1:i+2])
        isfin2 = np.isfinite(upper_Ep[i-1:i+2])
    
        if np.all(isfin1 == [False, True, False]):
            lower_Ep[i] = np.nan
        if np.all(isfin2 == [False, True, False]):
            upper_Ep[i] = np.nan
            
    return t[start_i:stop_j], lower_Ep, upper_Ep

In [ ]:
for _, row in df_dd.iterrows():
    fh = lib_search_dispersion.read_dmsp_flux_file(row.file)
    start_time = row.start_time
    end_time = row.end_time
    plot_spec(fh, start_time, end_time, method='dual_peaks')

In [ ]:
row = df_dd.iloc[1]
fh = lib_search_dispersion.read_dmsp_flux_file(row.file)
start_time = row.start_time
end_time = row.end_time
plot_spec(fh, start_time, end_time, axvline_idx=81)

In [ ]:
i = fh['t'].searchsorted(start_time)

data = fh['ion_d_ener'][:, i + 81]
smoothed = savgol_filter(data, 5, 2)
peaks = find_peaks(smoothed)[0]

plt.figure(figsize=(4, 3), dpi=200)
#plt.plot(fh['ch_energy'], data, '-', label='Data')
plt.plot(fh['ch_energy'], smoothed, '-')
plt.plot(fh['ch_energy'][peaks], smoothed[peaks], 'ro', markersize=10)
#peaks = np.array([peak for peak in peaks if data[peak] > 10**6.5 and np.log10(E[peak]) < 3.5])
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Energy eV')
plt.ylabel('Energy Flux')
plt.grid(color='gray', linestyle='dashed')
#plt.legend()